In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00


In [2]:
! pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.7 MB/s eta 0:00:00


In [3]:
!pip install transformers[torch] -U

In [1]:
from google.colab import files

uploaded = files.upload()


Saving resume.csv to resume (1).csv


In [1]:
import pandas as pd
df=pd.read_csv('resume.csv')


In [2]:
df.isnull().sum()

Category    0
Resume      0
dtype: int64

In [3]:
df.shape

(962, 2)

In [4]:
df=df.reset_index(drop=True)

In [5]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [6]:
df = df.rename(columns={'Category': 'label', 'Resume': 'target'})


In [7]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df)


In [8]:
ds

Dataset({
    features: ['label', 'target'],
    num_rows: 962
})

In [9]:
ds=ds.train_test_split(0.2)


In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'target'],
        num_rows: 769
    })
    test: Dataset({
        features: ['label', 'target'],
        num_rows: 193
    })
})

In [11]:
df['label'][30]

'Data Science'

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [21]:
def preprocess_function(examples):
    # Convert class labels to indices using label2id dictionary
    examples["label"] = [label2id[label] for label in examples["label"]]
    return tokenizer(examples["target"], truncation=True, padding=True)


In [22]:
tokenized_ds = ds.map(preprocess_function, batched=True)


Map:   0%|          | 0/769 [00:00<?, ? examples/s]

Map:   0%|          | 0/193 [00:00<?, ? examples/s]

In [23]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 769
    })
    test: Dataset({
        features: ['label', 'target', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 193
    })
})

In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [20]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


In [25]:
import evaluate

accuracy = evaluate.load("accuracy")


In [26]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)



In [20]:
# Mapping from class index to class label
class_index_to_label = {
    0: 'Data Science',
    1: 'HR',
    2: 'Advocate',
    3: 'Arts',
    4: 'Web Designing',
    5: 'Mechanical Engineer',
    6: 'Sales',
    7: 'Health and fitness',
    8: 'Civil Engineer',
    9: 'Java Developer',
    10: 'Business Analyst',
    11: 'SAP Developer',
    12: 'Automation Testing',
    13: 'Electrical Engineering',
    14: 'Operations Manager',
    15: 'Python Developer',
    16: 'DevOps Engineer',
    17: 'Network Security Engineer',
    18: 'PMO',
    19: 'Database',
    20: 'Hadoop',
    21: 'ETL Developer',
    22: 'DotNet Developer',
    23: 'Blockchain',
    24: 'Testing'
}

# Creating id2label and label2id dictionaries
id2label = dict(enumerate(class_index_to_label.values()))
label2id = {label: i for i, label in enumerate(class_index_to_label.values())}

# Print the dictionaries
print("id2label:", id2label)
print("label2id:", label2id)


id2label: {0: 'Data Science', 1: 'HR', 2: 'Advocate', 3: 'Arts', 4: 'Web Designing', 5: 'Mechanical Engineer', 6: 'Sales', 7: 'Health and fitness', 8: 'Civil Engineer', 9: 'Java Developer', 10: 'Business Analyst', 11: 'SAP Developer', 12: 'Automation Testing', 13: 'Electrical Engineering', 14: 'Operations Manager', 15: 'Python Developer', 16: 'DevOps Engineer', 17: 'Network Security Engineer', 18: 'PMO', 19: 'Database', 20: 'Hadoop', 21: 'ETL Developer', 22: 'DotNet Developer', 23: 'Blockchain', 24: 'Testing'}
label2id: {'Data Science': 0, 'HR': 1, 'Advocate': 2, 'Arts': 3, 'Web Designing': 4, 'Mechanical Engineer': 5, 'Sales': 6, 'Health and fitness': 7, 'Civil Engineer': 8, 'Java Developer': 9, 'Business Analyst': 10, 'SAP Developer': 11, 'Automation Testing': 12, 'Electrical Engineering': 13, 'Operations Manager': 14, 'Python Developer': 15, 'DevOps Engineer': 16, 'Network Security Engineer': 17, 'PMO': 18, 'Database': 19, 'Hadoop': 20, 'ETL Developer': 21, 'DotNet Developer': 22, '

In [27]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [28]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=25, id2label=id2label, label2id=label2id
).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
training_args = TrainingArguments(
    output_dir="./resume_screen",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    log_level="error",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.599580,0.502591
2,No log,1.837993,0.813472
3,No log,1.257232,0.917098
4,No log,0.839532,0.979275
5,No log,0.561874,0.979275
6,No log,0.416488,0.979275
7,No log,0.325922,0.979275
8,No log,0.274401,0.989637
9,No log,0.250583,0.989637
10,No log,0.241808,0.989637


TrainOutput(global_step=490, training_loss=1.0239507558394452, metrics={'train_runtime': 853.7546, 'train_samples_per_second': 9.007, 'train_steps_per_second': 0.574, 'total_flos': 2023741847746560.0, 'train_loss': 1.0239507558394452, 'epoch': 10.0})

In [30]:
trainer.save_model("./resume_screen_model1")

In [31]:
# Load the trained model
model = BertForSequenceClassification.from_pretrained("./resume_screen_model1")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Example text for inference
new_resume_text = "Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details \r\n\r\nData Science Assurance Associate \r\n\r\nData Science Assurance Associate - Ernst & Young LLP\r\nSkill Details \r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nPython- Exprience - 24 monthsCompany Details \r\ncompany - Ernst & Young LLP\r\ndescription - Fraud Investigations and Dispute Services   Assurance\r\nTECHNOLOGY ASSISTED REVIEW\r\nTAR (Technology Assisted Review) assists in accelerating the review process and run analytics and generate reports.\r\n* Core member of a team helped in developing automated review platform tool from scratch for assisting E discovery domain, this tool implements predictive coding and topic modelling by automating reviews, resulting in reduced labor costs and time spent during the lawyers review.\r\n* Understand the end to end flow of the solution, doing research and development for classification models, predictive analysis and mining of the information present in text data. Worked on analyzing the outputs and precision monitoring for the entire tool.\r\n* TAR assists in predictive coding, topic modelling from the evidence by following EY standards. Developed the classifier models in order to identify 'red flags' and fraud-related issues.\r\n\r\nTools & Technologies: Python, scikit-learn, tfidf, word2vec, doc2vec, cosine similarity, NaÃ¯ve Bayes, LDA, NMF for topic modelling, Vader and text blob for sentiment analysis. Matplot lib, Tableau dashboard for reporting.\r\n\r\nMULTIPLE DATA SCIENCE AND ANALYTIC PROJECTS (USA CLIENTS)\r\nTEXT ANALYTICS - MOTOR VEHICLE CUSTOMER REVIEW DATA * Received customer feedback survey data for past one year. Performed sentiment (Positive, Negative & Neutral) and time series analysis on customer comments across all 4 categories.\r\n* Created heat map of terms by survey category based on frequency of words * Extracted Positive and Negative words across all the Survey categories and plotted Word cloud.\r\n* Created customized tableau dashboards for effective reporting and visualizations.\r\nCHATBOT * Developed a user-friendly chatbot for one of our Products which handles simple questions about hours of operation, reservation options, and so on.\r\n* This chatbot serves entire product-related questions. Giving an overview of the tool via QA platform and also gives recommendation responses so that user question to build a chain of relevant answers.\r\n* This too has intelligence to build the pipeline of questions as per user requirements and asks the relevant /recommended questions.\r\n\r\nTools & Technologies: Python, Natural language processing, NLTK, spacy, topic modelling, Sentiment analysis, Word Embedding, scikit-learn, JavaScript/JQuery, SqlServer\r\n\r\nINFORMATION GOVERNANCE\r\nOrganizations make informed decisions about all of the information they store. The integrated Information Governance portfolio synthesizes intelligence across unstructured data sources and facilitates action to ensure organizations are best positioned to counter information risk.\r\n* Scan data from multiple sources of formats and parse different file formats, extract Meta data information, push results for indexing elastic search and created customized, interactive dashboards using kibana.\r\n* Performing ROT Analysis on the data which gives information about data which helps identify content that is either Redundant, Outdated, or Trivial.\r\n* Performing full-text search analysis on elastic search with predefined methods which can tag as (PII) personally identifiable information (social security numbers, addresses, names, etc.) which frequently targeted during cyber-attacks.\r\nTools & Technologies: Python, Flask, Elastic Search, Kibana\r\n\r\nFRAUD ANALYTIC PLATFORM\r\nFraud Analytics and investigative platform to review all red flag cases.\r\nâ\x80¢ FAP is a Fraud Analytics and investigative platform with inbuilt case manager and suite of Analytics for various ERP systems.\r\n* It can be used by clients to interrogate their Accounting systems for identifying the anomalies which can be indicators of fraud by running advanced analytics\r\nTools & Technologies: HTML, JavaScript, SqlServer, JQuery, CSS, Bootstrap, Node.js, D3.js, DC.js"

# Tokenize and encode the input text
inputs = tokenizer(new_resume_text, return_tensors="pt", truncation=True, padding=True)

# Forward pass to get predictions
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted class
predicted_class = torch.argmax(outputs.logits).item()

# Map the predicted class index to the label
predicted_label = id2label[predicted_class]

# Print the result
print(f"Predicted Label: {predicted_label}")

Predicted Label: Data Science
